In [1]:
!pip install tikzplotlib


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Function to calculate the distance from a point to a line given by ax + by + d = 0
def point_to_line_dist(params, points):
    a, b, d = params
    norm_ab = np.sqrt(a**2 + b**2)
    dists = np.abs(a * points[:, 0] + b * points[:, 1] + d) / norm_ab
    return dists

# RANSAC algorithm to fit a line to the data
def ransac_fit_line(data_points, max_iters=1000, distance_threshold=0.6, min_inlier_count=35):
    optimal_model, optimal_inliers = None, []
    for _ in range(max_iters):
        # Select two random points to define a line
        pt1, pt2 = data_points[np.random.choice(len(data_points), 2, replace=False)]

        # Calculate the line parameters (ax + by + d = 0) using the two points
        a, b, d = pt2[1] - pt1[1], pt1[0] - pt2[0], -(pt2[1] - pt1[1]) * pt1[0] - (pt1[0] - pt2[0]) * pt1[1]

        # Normalize to ensure ||[a, b]|| = 1
        norm_factor = np.hypot(a, b)
        a, b, d = a / norm_factor, b / norm_factor, d / norm_factor

        # Compute distances of all points from the line
        dist_from_line = np.abs(a * data_points[:, 0] + b * data_points[:, 1] + d)
        inliers = data_points[dist_from_line < distance_threshold]

        # Update if this model has more inliers
        if len(inliers) > len(optimal_inliers) and len(inliers) >= min_inlier_count:
            optimal_inliers, optimal_model = inliers, [a, b, d]

    return optimal_model, optimal_inliers

# Function to calculate the radial distance from a point to a circle
def point_to_circle_dist(params, points):
    x_center, y_center, radius = params
    dists = np.abs(np.sqrt((points[:, 0] - x_center)**2 + (points[:, 1] - y_center)**2) - radius)
    return dists

# RANSAC algorithm to fit a circle to the data
def ransac_fit_circle(points, max_iters=1000, error_threshold=1.5, min_inliers=45):
    optimal_circle_params = None
    max_inliers = []
    
    for _ in range(max_iters):
        # Randomly pick 3 points to define the circle
        random_sample = np.random.choice(len(points), 3, replace=False)
        p1, p2, p3 = points[random_sample]
        
        # Formulate and solve the system of equations to get circle parameters
        matrix_a = np.array([
            [p1[0], p1[1], 1],
            [p2[0], p2[1], 1],
            [p3[0], p3[1], 1]
        ])
        vector_b = np.array([
            [-(p1[0]**2 + p1[1]**2)],
            [-(p2[0]**2 + p2[1]**2)],
            [-(p3[0]**2 + p3[1]**2)]
        ])
        
        try:
            # Solve for the circle parameters
            params_solution = np.linalg.solve(matrix_a, vector_b)
            x_center = -0.5 * params_solution[0][0]
            y_center = -0.5 * params_solution[1][0]
            radius = np.sqrt((x_center**2 + y_center**2) - params_solution[2][0])
        except np.linalg.LinAlgError:
            continue
        
        # Calculate radial distances
        radial_errors = point_to_circle_dist([x_center, y_center, radius], points)
        
        # Find inliers based on the radial distance
        inliers = points[radial_errors < error_threshold]
        
        # Update if this model has more inliers
        if len(inliers) > len(max_inliers) and len(inliers) >= min_inliers:
            max_inliers = inliers
            optimal_circle_params = [x_center, y_center, radius]
    
    return optimal_circle_params, max_inliers

# Generate synthetic dataset (a circle and a line with noise)
np.random.seed(0)
total_points = 100
half = total_points // 2
radius_val = 10
true_center_x, true_center_y = 2, 3
noise_factor = radius_val / 16
theta_vals = np.random.uniform(0, 2 * np.pi, half)
noise = noise_factor * np.random.randn(half)
x_circle = true_center_x + (radius_val + noise) * np.cos(theta_vals)
y_circle = true_center_y + (radius_val + noise) * np.sin(theta_vals)
circle_data = np.hstack((x_circle.reshape(half, 1), y_circle.reshape(half, 1)))

# Generate noisy line data
slope, intercept = -1, 2
x_line = np.linspace(-12, 12, half)
y_line = slope * x_line + intercept + 1.0 * np.random.randn(half)
line_data = np.hstack((x_line.reshape(half, 1), y_line.reshape(half, 1)))

# Combine circle and line data
all_data = np.vstack((circle_data, line_data))

# Apply RANSAC to estimate the line
best_line_model, line_inliers = ransac_fit_line(all_data)

# Exclude line inliers and estimate the circle
remaining_points = np.array([pt for pt in all_data if pt not in line_inliers])
best_circle_model, circle_inliers = ransac_fit_circle(remaining_points)

# Plotting the results
fig, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=300)

# Original points
ax.scatter(line_data[:, 0], line_data[:, 1], label='Line Points')
ax.scatter(circle_data[:, 0], circle_data[:, 1], label='Circle Points')

# Ground truth circle and line
ground_truth_circle = plt.Circle((true_center_x, true_center_y), radius_val, color='limegreen', fill=False, label='True Circle')
ax.add_patch(ground_truth_circle)
x_limits, y_limits = ax.get_xlim()
x_range = np.array([x_limits, y_limits])
y_range = slope * x_range + intercept
plt.plot(x_range, y_range, color='m', label='True Line')

# Plot estimated line
a_coeff, b_coeff, d_coeff = best_line_model
x_plot_vals = np.array(ax.get_xlim())
y_plot_vals = -(a_coeff * x_plot_vals + d_coeff) / b_coeff
ax.plot(x_plot_vals, y_plot_vals, 'r-', label='Estimated Line')

# Plot estimated circle
center_x, center_y, estimated_radius = best_circle_model
estimated_circle = plt.Circle((center_x, center_y), estimated_radius, color='blue', fill=False, label='Estimated Circle')
ax.add_patch(estimated_circle)

# Highlight the inliers
ax.scatter(line_inliers[:, 0], line_inliers[:, 1], color='red', label='Line Inliers')
ax.scatter(circle_inliers[:, 0], circle_inliers[:, 1], color='blue', label='Circle Inliers')

# Show the final plot
ax.legend()
plt.show()


ImportError: cannot import name 'common_texification' from 'matplotlib.backends.backend_pgf' (c:\Users\herat\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\backends\backend_pgf.py)